- 학번 : 201611125
- 이름 : 김정수
- 학과 : 게임전공

## 문제 ZSCORE, CDF 계산

성적데이터는 n이 적지만, 정규분포를 이룬다고 가정하자.

* 1-1 성적데이터로 DataFrame을 생성.

* 1-2 zscore 컬럼을 생성.

* 1-3 cdf 컬럼을 생성.

In [23]:
import pyspark
import os
from pyspark.sql.types import *
from pyspark.sql.functions import udf
import numpy as np
from pyspark.sql import functions as F
from scipy.stats import norm

In [24]:
myConf = pyspark.SparkConf()
spark = pyspark.sql.SparkSession.builder\
        .master("local")\
        .appName("myApp")\
        .config(conf = myConf)\
        .getOrCreate()

## 문제 1-1 성적데이터로 DataFrame을 생성

In [25]:
# 사용할 성적 데이터
marks=["김하나, English, 100",
    "김하나, Math, 80",
    "임하나, English, 70",
    "임하나, Math, 100",
    "김갑돌, English, 82.3",
    "김갑돌, Math, 98.5"]

In [26]:
# RDD로 먼저 생성 후
myRDD = spark.sparkContext.parallelize(marks)
myRDD = myRDD.map(lambda x: x.split(", ")).collect()

In [27]:
# 데이터프레임으로 변환
gradeDF = spark.createDataFrame(myRDD, ["name", "subject", "grade"])
gradeDF.show()

+------+-------+-----+
|  name|subject|grade|
+------+-------+-----+
|김하나|English|  100|
|김하나|   Math|   80|
|임하나|English|   70|
|임하나|   Math|  100|
|김갑돌|English| 82.3|
|김갑돌|   Math| 98.5|
+------+-------+-----+



## 문제 1-2 zscore 컬럼을 생성
- zscore는 각 측정 값을 평균에서 빼고난 후 표준편차로 나누어 계산
- zscore = (점수 - 평균) / 표준 편차
- zscore를 계산하려면, 평균과 표준편차를 알아야 한다.
- 계산식에 F함수를 직접 사용하면 오류가 발생한다. 따로 평균과 표준편차를 구해서 계산식에서 사용해야 한다.

In [28]:
# zscore 계산 전, grade 컬럼의 데이터 타입을 float로 변경
gradeDF = gradeDF.withColumn("grade_float", gradeDF["grade"].cast(FloatType()))
gradeDF.show()

+------+-------+-----+-----------+
|  name|subject|grade|grade_float|
+------+-------+-----+-----------+
|김하나|English|  100|      100.0|
|김하나|   Math|   80|       80.0|
|임하나|English|   70|       70.0|
|임하나|   Math|  100|      100.0|
|김갑돌|English| 82.3|       82.3|
|김갑돌|   Math| 98.5|       98.5|
+------+-------+-----+-----------+



In [33]:
# 평균과 표준편차 따로 계산
ms = gradeDF.agg({"grade_float" : "avg"})
_avg = ms.rdd.flatMap(list).first()

ss = gradeDF.agg({"grade_float" : "std"})
_std = ss.rdd.flatMap(list).first()

In [37]:
# zscore = (grade - mean) / std
zscoreUdf = F.udf(lambda x: (x - _avg) / _std, FloatType())
gradeDF = gradeDF.withColumn("zscore", zscoreUdf(gradeDF["grade_float"]))

In [38]:
gradeDF.show()

+------+-------+-----+-----------+-----------+
|  name|subject|grade|grade_float|     zscore|
+------+-------+-----+-----------+-----------+
|김하나|English|  100|      100.0|  0.9020148|
|김하나|   Math|   80|       80.0| -0.6621728|
|임하나|English|   70|       70.0| -1.4442666|
|임하나|   Math|  100|      100.0|  0.9020148|
|김갑돌|English| 82.3|       82.3|-0.48229098|
|김갑돌|   Math| 98.5|       98.5| 0.78470075|
+------+-------+-----+-----------+-----------+



## 문제 1-3 cdf 컬럼을 생성

In [39]:
# scipy 라이브러리의 norm.cdf 사용
rv = norm(loc=0, scale=1)
cdfUdf = udf(lambda x: float(rv.cdf(x)), FloatType())

In [40]:
gradeDF = gradeDF.withColumn("cdf", cdfUdf(gradeDF["zscore"]))

In [41]:
gradeDF.show()

+------+-------+-----+-----------+-----------+----------+
|  name|subject|grade|grade_float|     zscore|       cdf|
+------+-------+-----+-----------+-----------+----------+
|김하나|English|  100|      100.0|  0.9020148| 0.8164755|
|김하나|   Math|   80|       80.0| -0.6621728|0.25393024|
|임하나|English|   70|       70.0| -1.4442666|  0.074332|
|임하나|   Math|  100|      100.0|  0.9020148| 0.8164755|
|김갑돌|English| 82.3|       82.3|-0.48229098|0.31479964|
|김갑돌|   Math| 98.5|       98.5| 0.78470075|0.78368545|
+------+-------+-----+-----------+-----------+----------+

